# Illustrative Examples
In this notebook, we will solve some problems and use them as examples to illustrate features of Cadabra.

## Wildcards and independet indices
When defining an operation, such as the covariant derivative, Cadabra does not explicitly know how to handle indices up and indices down, and will sometimes switch between them without properly applying the metric, and/or leading to less desireable equation forms ( a connection with three up indices ). 

Another behaviour by default in Cadabra, is that the rules we define are explicit unless we use a wildcard operation `?` in our argument; that is to say
```
A^{a} -> B^{a}
```
will only transform the tensor symbol $A^a$ into $B^a$, whereas
```
A?^{a} -> B^{a}
```
will transform any $\,^1_0$ tensor into $B^a$. Let us illustrate an example.

We define our indices as being `independent`, and through this tell Cadabra that we want strict control over the raising and lowering operations:

In [1]:
{a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u#}::Indices(position=independent).

\nabla{#}::Derivative.
\partial{#}::PartialDerivative.

Next, we create a single replacement rule any $\,^1_0$ or $\,^0_1$ covariant derivative. We do this using a list, which we can call with `repeat=True` in `substitute`:

In [2]:
deriv1 := \nabla_{a}{A?^{b}} -> \partial_{a}{A?^{b}}
    + \Gamma^{b}_{c a} A?^{c}.

deriv2 := \nabla_{a}{A?_{b}} -> \partial_{a}{A?_{b}}
    - \Gamma^{c}_{b a} A?_{c}.

deriv := {@(deriv1), @(deriv2)};

${}\left[\nabla_{a}\left(A?^{b}\right) \rightarrow \partial_{a}\left(A?^{b}\right)+\Gamma^{b}\,_{c a} A?^{c},~ \nabla_{a}\left(A?_{b}\right) \rightarrow \partial_{a}\left(A?_{b}\right)-\Gamma^{c}\,_{b a} A?_{c}\right]$

Now we define the expression we wish to act on:

In [3]:
uv := \nabla_{a}{v_{b} u^{b}} - \partial_{a}{v_{b} u^{b}};

${}\nabla_{a}\left(v_{b} u^{b}\right)-\partial_{a}\left(v_{b} u^{b}\right)$

Because we told Cadabra that $\nabla$ and $\partial$ act as derivatives, we can use an algorithm like `product_rule` to expand the expression:

In [4]:
product_rule(uv);

${}\nabla_{a}{v_{b}} u^{b}+v_{b} \nabla_{a}{u^{b}}-\partial_{a}{v_{b}} u^{b}-v_{b} \partial_{a}{u^{b}}$

Then we can apply our regular substitution and distributions:

In [5]:
substitute(uv, deriv, repeat=True);
distribute(uv);

${}\left(\partial_{a}{v_{b}}-\Gamma^{c}\,_{b a} v_{c}\right) u^{b}+v_{b} \left(\partial_{a}{u^{b}}+\Gamma^{b}\,_{c a} u^{c}\right)-\partial_{a}{v_{b}} u^{b}-v_{b} \partial_{a}{u^{b}}$

${}-\Gamma^{c}\,_{b a} v_{c} u^{b}+v_{b} \Gamma^{b}\,_{c a} u^{c}$

Finally we use `sort_product` to arrange the result in the Cadabra sort hierarchy, and call `rename_dummies` to simplify the expression:

In [6]:
sort_product(uv);
rename_dummies(uv);

${}-\Gamma^{c}\,_{b a} u^{b} v_{c}+\Gamma^{b}\,_{c a} u^{c} v_{b}$

${}0$

## Assembling substitution rules
Let us now use an example which requires many rules for substitution and expansion; consider we want to show
$$
\nabla_{a}{v_{b c}} = \partial_{a}{v_{b c}} - \Gamma^{d}_{b a} v_{d c} - \Gamma^{d}_{c a} v_{b d}.
$$

To do this, we define the rules:

In [7]:
derivU := \nabla_{a}{A?^{b}} -> 
    \partial_{a}{A?^{b}} + \Gamma^{b}_{c a} A?^{c}.
derivD := \nabla_{a}{A?_{b}} -> 
    \partial_{a}{A?_{b}} - \Gamma^{c}_{b a} A?_{c}.

deriv := {@(derivU), @(derivD)};

vab := v_{a b} -> A_{a} B_{b};
iab := A_{a} B_{b} -> v_{a b};

pab := \partial_{a}{A_{b}} B_{c} -> \partial_{a}{A_{b} B_{c}} - A_{b} \partial_{a}{B_{c}};

${}\left[\nabla_{a}\left(A?^{b}\right) \rightarrow \partial_{a}\left(A?^{b}\right)+\Gamma^{b}\,_{c a} A?^{c},~ \nabla_{a}\left(A?_{b}\right) \rightarrow \partial_{a}\left(A?_{b}\right)-\Gamma^{c}\,_{b a} A?_{c}\right]$

${}v_{a b} \rightarrow A_{a} B_{b}$

${}A_{a} B_{b} \rightarrow v_{a b}$

${}\partial_{a}{A_{b}} B_{c} \rightarrow \partial_{a}\left(A_{b} B_{c}\right)-A_{b} \partial_{a}{B_{c}}$

Then we create our problem expression:

In [8]:
Dvab := \nabla_{a}{v_{b c}};

${}\nabla_{a}{v_{b c}}$

And apply our rules:

In [9]:
substitute(Dvab, vab);
product_rule(Dvab);
substitute(Dvab, deriv, repeat=True);
distribute(Dvab);
substitute(Dvab, pab);
canonicalise(Dvab);
substitute(Dvab, iab);
sort_product(Dvab);

${}\nabla_{a}\left(A_{b} B_{c}\right)$

${}\nabla_{a}{A_{b}} B_{c}+A_{b} \nabla_{a}{B_{c}}$

${}\left(\partial_{a}{A_{b}}-\Gamma^{d}\,_{b a} A_{d}\right) B_{c}+A_{b} \left(\partial_{a}{B_{c}}-\Gamma^{d}\,_{c a} B_{d}\right)$

${}\partial_{a}{A_{b}} B_{c}-\Gamma^{d}\,_{b a} A_{d} B_{c}+A_{b} \partial_{a}{B_{c}}-A_{b} \Gamma^{d}\,_{c a} B_{d}$

${}\partial_{a}\left(A_{b} B_{c}\right)-\Gamma^{d}\,_{b a} A_{d} B_{c}-A_{b} \Gamma^{d}\,_{c a} B_{d}$

${}\partial_{a}\left(A_{b} B_{c}\right)-\Gamma^{d}\,_{b a} A_{d} B_{c}-A_{b} \Gamma^{d}\,_{c a} B_{d}$

${}\partial_{a}{v_{b c}}-\Gamma^{d}\,_{b a} v_{d c}-v_{b d} \Gamma^{d}\,_{c a}$

${}\partial_{a}{v_{b c}}-\Gamma^{d}\,_{b a} v_{d c}-\Gamma^{d}\,_{c a} v_{b d}$

What if we changed our prescription to:

In [10]:
Dvab := \nabla_{a}{v^{b}_{c}};

${}\nabla_{a}{v^{b}\,_{c}}$

Now we need to add the rules:

In [11]:
vab := v^{a}_{b} -> A^{a} B_{b};
iab := A^{a} B_{b} -> v^{a}_{b};

pab := \partial_{a}{A^{b}} B_{c} -> \partial_{a}{A^{b} B_{c}} - A^{b} \partial_{a}{B_{c}};

${}v^{a}\,_{b} \rightarrow A^{a} B_{b}$

${}A^{a} B_{b} \rightarrow v^{a}\,_{b}$

${}\partial_{a}{A^{b}} B_{c} \rightarrow \partial_{a}\left(A^{b} B_{c}\right)-A^{b} \partial_{a}{B_{c}}$

These are very similar to our previous rules, however for the case where we have an index up. We can still apply the same solving method once we have our new rules in place:

In [12]:
substitute(Dvab, vab);
product_rule(Dvab);
substitute(Dvab, deriv, repeat=True);
distribute(Dvab);
substitute(Dvab, pab);
canonicalise(Dvab);
substitute(Dvab, iab);
sort_product(Dvab);

${}\nabla_{a}\left(A^{b} B_{c}\right)$

${}\nabla_{a}{A^{b}} B_{c}+A^{b} \nabla_{a}{B_{c}}$

${}\left(\partial_{a}{A^{b}}+\Gamma^{b}\,_{d a} A^{d}\right) B_{c}+A^{b} \left(\partial_{a}{B_{c}}-\Gamma^{d}\,_{c a} B_{d}\right)$

${}\partial_{a}{A^{b}} B_{c}+\Gamma^{b}\,_{d a} A^{d} B_{c}+A^{b} \partial_{a}{B_{c}}-A^{b} \Gamma^{d}\,_{c a} B_{d}$

${}\partial_{a}\left(A^{b} B_{c}\right)+\Gamma^{b}\,_{d a} A^{d} B_{c}-A^{b} \Gamma^{d}\,_{c a} B_{d}$

${}\partial_{a}\left(A^{b} B_{c}\right)+\Gamma^{b}\,_{d a} A^{d} B_{c}-A^{b} \Gamma^{d}\,_{c a} B_{d}$

${}\partial_{a}{v^{b}\,_{c}}+\Gamma^{b}\,_{d a} v^{d}\,_{c}-v^{b}\,_{d} \Gamma^{d}\,_{c a}$

${}\partial_{a}{v^{b}\,_{c}}+\Gamma^{b}\,_{d a} v^{d}\,_{c}-\Gamma^{d}\,_{c a} v^{b}\,_{d}$

## Chaining rules together
We can create quite complex and involved rules by calling `substitute` on themselves. Consider the first and second covariant derivative:

In [13]:
deriv1 := \nabla_{a}{v^{b}} -> \partial_{a}{v^{b}}
    + \Gamma^{b}_{d a} v^{d};

deriv2 := \nabla_{a}{\nabla_{b}{v^{c}}} -> 
    \partial_{a}{\nabla_{b}{v^{c}}}
    + \Gamma^{c}_{d a} \nabla_{b}{v^{d}}
    - \Gamma^{d}_{b a} \nabla_{d}{v^{c}};

${}\nabla_{a}{v^{b}} \rightarrow \partial_{a}{v^{b}}+\Gamma^{b}\,_{d a} v^{d}$

${}\nabla_{a}\left(\nabla_{b}{v^{c}}\right) \rightarrow \partial_{a}\left(\nabla_{b}{v^{c}}\right)+\Gamma^{c}\,_{d a} \nabla_{b}{v^{d}}-\Gamma^{d}\,_{b a} \nabla_{d}{v^{c}}$

We can combine them as such:

In [14]:
hold := @(deriv2).
substitute(hold,deriv1);

${}\nabla_{a}\left(\partial_{b}{v^{c}}+\Gamma^{c}\,_{d b} v^{d}\right) \rightarrow \partial_{a}\left(\partial_{b}{v^{c}}+\Gamma^{c}\,_{d b} v^{d}\right)+\Gamma^{c}\,_{d a} \left(\partial_{b}{v^{d}}+\Gamma^{d}\,_{e b} v^{e}\right)-\Gamma^{d}\,_{b a} \left(\partial_{d}{v^{c}}+\Gamma^{c}\,_{e d} v^{e}\right)$

But this creates a problem; the call to substitute also replaced the argument of our rule on the LHS. We can avoid this by crafting an intermediary expression to substitute through:

In [15]:
expr := v^{c}_{b a} -> \nabla_{a}{\nabla_{b}{v^{c}}};
save := @(expr).

${}v^{c}\,_{b a} \rightarrow \nabla_{a}\left(\nabla_{b}{v^{c}}\right)$

In [16]:
substitute(expr,deriv2);
substitute(expr,deriv1);
distribute(expr);
product_rule(expr);
canonicalise(expr);

${}v^{c}\,_{b a} \rightarrow \partial_{a}\left(\nabla_{b}{v^{c}}\right)+\Gamma^{c}\,_{d a} \nabla_{b}{v^{d}}-\Gamma^{d}\,_{b a} \nabla_{d}{v^{c}}$

${}v^{c}\,_{b a} \rightarrow \partial_{a}\left(\partial_{b}{v^{c}}+\Gamma^{c}\,_{d b} v^{d}\right)+\Gamma^{c}\,_{d a} \left(\partial_{b}{v^{d}}+\Gamma^{d}\,_{e b} v^{e}\right)-\Gamma^{d}\,_{b a} \left(\partial_{d}{v^{c}}+\Gamma^{c}\,_{e d} v^{e}\right)$

${}v^{c}\,_{b a} \rightarrow \partial_{a b}{v^{c}}+\partial_{a}\left(\Gamma^{c}\,_{d b} v^{d}\right)+\Gamma^{c}\,_{d a} \partial_{b}{v^{d}}+\Gamma^{c}\,_{d a} \Gamma^{d}\,_{e b} v^{e}-\Gamma^{d}\,_{b a} \partial_{d}{v^{c}}-\Gamma^{d}\,_{b a} \Gamma^{c}\,_{e d} v^{e}$

${}v^{c}\,_{b a} \rightarrow \partial_{a b}{v^{c}}+\partial_{a}{\Gamma^{c}\,_{d b}} v^{d}+\Gamma^{c}\,_{d b} \partial_{a}{v^{d}}+\Gamma^{c}\,_{d a} \partial_{b}{v^{d}}+\Gamma^{c}\,_{d a} \Gamma^{d}\,_{e b} v^{e}-\Gamma^{d}\,_{b a} \partial_{d}{v^{c}}-\Gamma^{d}\,_{b a} \Gamma^{c}\,_{e d} v^{e}$

${}v^{c}\,_{b a} \rightarrow \partial_{a b}{v^{c}}+\partial_{a}{\Gamma^{c}\,_{d b}} v^{d}+\Gamma^{c}\,_{d b} \partial_{a}{v^{d}}+\Gamma^{c}\,_{d a} \partial_{b}{v^{d}}+\Gamma^{c}\,_{d a} \Gamma^{d}\,_{e b} v^{e}-\Gamma^{d}\,_{b a} \partial_{d}{v^{c}}-\Gamma^{c}\,_{d e} \Gamma^{e}\,_{b a} v^{d}$

And use our saved expression to replace the LHS without fear:

In [17]:
substitute(expr, save);

${}\nabla_{a}\left(\nabla_{b}{v^{c}}\right) \rightarrow \partial_{a b}{v^{c}}+\partial_{a}{\Gamma^{c}\,_{d b}} v^{d}+\Gamma^{c}\,_{d b} \partial_{a}{v^{d}}+\Gamma^{c}\,_{d a} \partial_{b}{v^{d}}+\Gamma^{c}\,_{d a} \Gamma^{d}\,_{e b} v^{e}-\Gamma^{d}\,_{b a} \partial_{d}{v^{c}}-\Gamma^{c}\,_{d e} \Gamma^{e}\,_{b a} v^{d}$

## Curvature across a scalar field
We want to show that 

$$
\left( \nabla_a \nabla_b - \nabla_b \nabla_a \right) \phi = \left( \Gamma^{c}_{a b} -\Gamma^{c}_{b a} \right) \partial_{c}\phi.
$$

For scalars, we define the covariant derivative:

In [18]:
dphi := \nabla_{a}{\phi} -> \partial_{a}{\phi};

${}\nabla_{a}{\phi} \rightarrow \partial_{a}{\phi}$

In [19]:
deriv := \nabla_{a}{A?_{b}} -> \partial_{a}{A?_{b}} - \Gamma^{c}_{b a} A?_{c};

${}\nabla_{a}\left(A?_{b}\right) \rightarrow \partial_{a}\left(A?_{b}\right)-\Gamma^{c}\,_{b a} A?_{c}$

In [20]:
expr := \nabla_{a}{\nabla_{b}{\phi}}
    - \nabla_{b}{\nabla_{a}{\phi}};

${}\nabla_{a}\left(\nabla_{b}{\phi}\right)-\nabla_{b}\left(\nabla_{a}{\phi}\right)$

In [21]:
substitute(expr, dphi);

${}\nabla_{a}\left(\partial_{b}{\phi}\right)-\nabla_{b}\left(\partial_{a}{\phi}\right)$

Since we know $\partial_a \phi$ is a tensor quantity, we will use anonymous functions to replace and reinsert the quantity, so that we can act with our covariant derivative correctly:

In [22]:
substitute(expr, $\partial_{a}{\phi} -> w_{a}$);
substitute(expr, deriv);
substitute(expr, $w_{a} -> \partial_{a}{\phi}$);

${}\nabla_{a}{w_{b}}-\nabla_{b}{w_{a}}$

${}\partial_{a}{w_{b}}-\Gamma^{c}\,_{b a} w_{c}-\partial_{b}{w_{a}}+\Gamma^{c}\,_{a b} w_{c}$

${}\partial_{a b}{\phi}-\Gamma^{c}\,_{b a} \partial_{c}{\phi}-\partial_{b a}{\phi}+\Gamma^{c}\,_{a b} \partial_{c}{\phi}$

Finally, we can canonicalise the result:

In [23]:
canonicalise(expr);

${}-\Gamma^{c}\,_{b a} \partial_{c}{\phi}+\Gamma^{c}\,_{a b} \partial_{c}{\phi}$

In [24]:
kill := \Gamma^{a}_{b c} -> 0;

Gamma := \Gamma^{a}_{b c}
       + x^{d} \partial_{d}{\Gamma^{a}_{b c}};

substitute (Gamma,kill);


${}\Gamma^{a}\,_{b c} \rightarrow 0$

${}\Gamma^{a}\,_{b c}+x^{d} \partial_{d}{\Gamma^{a}\,_{b c}}$

${}0$